In [4]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor

In [14]:
data = pd.read_csv("Airplane.csv")
# خواندن فایل

data = data.dropna()  # حذف دیتا های خالی

# تبدیل مقادیر غیر عددی به عددی
data["satisfaction"] = data["satisfaction"].map(
    {"neutral or dissatisfied": 0, "satisfied": 1}).astype(int)
data["Customer Type"] = data["Customer Type"].map(
    {"disloyal Customer": 0, "Loyal Customer": 1}).astype(int)
data["Type of Travel"] = data["Type of Travel"].map(
    {"Personal Travel": 0, "Business travel": 1}).astype(int)
data["Gender"] = data["Gender"].map({"Female": 0, "Male": 1}).astype(int)
data["Class"] = data["Class"].map(
    {"Eco": 0, "Eco Plus": 1, "Business": 2}).astype(int)

# دسته بندی داده های پیوسته
# Arrival Delay in Minutes
data.loc[data["Arrival Delay in Minutes"] <= 5, "Arrival Delay in Minutes"] = 0
data.loc[(data["Arrival Delay in Minutes"] > 5),
         "Arrival Delay in Minutes"] = 1

# Age
data.loc[data["Age"] <= 20, "Age"] = 0
data.loc[(data["Age"] > 20) & (data["Age"] <= 39), "Age"] = 1
data.loc[(data["Age"] > 39) & (data["Age"] <= 60), "Age"] = 2
data.loc[(data["Age"] > 60), "Age"] = 3

# Cleanliness
data.loc[data["Cleanliness"] < 3, "Cleanliness"] = 0
data.loc[data["Cleanliness"] == 3, "Cleanliness"] = 1
data.loc[(data["Cleanliness"] > 3), "Cleanliness"] = 2

# Flight Distance
data.loc[data["Flight Distance"] <= 1000, "Flight Distance"] = 0
data.loc[(data["Flight Distance"] > 1000) & (
    data["Flight Distance"] <= 2000), "Flight Distance"] = 1
data.loc[(data["Flight Distance"] > 2000) & (
    data["Flight Distance"] <= 3000), "Flight Distance"] = 2
data.loc[(data["Flight Distance"] > 3000), "Flight Distance"] = 3

# Departure Delay in Minutes
data.loc[data["Departure Delay in Minutes"]
         <= 5, "Departure Delay in Minutes"] = 0
data.loc[(data["Departure Delay in Minutes"] > 5) & (
    data["Departure Delay in Minutes"] <= 25), "Departure Delay in Minutes"] = 1
data.loc[(data["Departure Delay in Minutes"] > 25),
         "Departure Delay in Minutes"] = 2

# انتخاب 10000 داده ابتدایی به عنوان داده تست
test = data.tail(10000)

# حذف 10000 داده ابتدایی از دیتا فریم
data = data.head(90000)

# جدا سازی داده های {satisfied و neutral or dissatisfied}
satisfaction_0 = data[data['satisfaction'] == 0]
satisfaction_1 = data[data['satisfaction'] == 1]
random.seed(43)

# انتخاب تعداد 50 داده رندوم از هر گروه
satisfaction_0_random = random.sample(satisfaction_0.index.tolist(), 10000)
satisfaction_1_random = random.sample(satisfaction_1.index.tolist(), 10000)

# اتصال این دو دسته دیتا
data = pd.concat([data.loc[satisfaction_0_random],
                 data.loc[satisfaction_1_random]])

# حذف ستون های اضافه
data = data.drop(["index", "id", "Gender"], axis=1)
test = test.drop(["index", "id", "Gender"], axis=1)
# جنسیت به دلیل نداشتن تاثیر از داده ها حذف شده

In [12]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

y = data['satisfaction'];
x = data.drop(["satisfaction"], axis=1);

# تقسیم داده به دو بخش آموزش و آزمون
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42)

# نرمال‌سازی داده‌ها
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ساخت و آموزش مدل
model = MLPRegressor(hidden_layer_sizes=(100, 50),
                     max_iter=500, random_state=42)
model.fit(X_train_scaled, y_train)

# ارزیابی مدل
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 0.04765577012398413


In [15]:
from sklearn.metrics import accuracy_score


def compare_y_test_and_y_pred(y_test, y_pred, threshold=0.5):
    # تبدیل مقادیر پیش‌بینی به دسته‌بندی
    y_pred_class = (y_pred > threshold).astype(int)

    # محاسبه دقت
    accuracy = accuracy_score(y_test, y_pred_class)
    print(f'Accuracy: {accuracy * 100:.2f}%')


# مثال استفاده
compare_y_test_and_y_pred(y_test, y_pred)

Accuracy: 94.60%
